## Práctica 2. Procesamiento de lenguaje natural

### Parte 1. Análisis de sentimientos

In [151]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [152]:
with open("yelp_labelled.txt","r") as text_file:
    lines_org = text_file.read().split('\n')
len(lines)

1000

In [153]:
lines = [line.split("\t") for line in lines_org if len(line.split("\t"))==2 and line.split("\t")[1]!='']
len(lines)

1000

### Apartado a)

#### Configura una partición train-test usando el 75% de los datos para entrenamiento y el 25% restante para test.

In [154]:
train_data = [line[0] for line in lines ]
labels = [int(line[1]) for line in lines]
X_train = data[:750]
X_test = data[750:]
y_train = labels[:750]
y_test = labels[750:]



#### Vamos a estudiar varias representaciones de bolsa de palabras, pero todas ellas utilizarán countVectorizer con el diccionario que se crea a partir de los términos del propio corpus y la lista de palabras vacías (stop_words) que proporciona sklearn para el inglés. Las 4 posibilidades que estudiaremos surgen de combinar los siguientes 2 parámetros:

    - Bolsa de palabras binaria y bolsa de palabras con TF/IDF (parámetro binary).
    
    - Usando un rango de n-gramas de (1,1) y de (1,2) (parámetro ngram_range).

In [155]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [156]:
vectorizer = CountVectorizer(stop_words='english')
# Tomamos los textos del conjunto de entrenamiento y los transformamos en 
# una matriz de datos (palabras) según el diccionario estándar
train_vector_data=vectorizer.fit_transform(X_train)

In [157]:
feature_names = vectorizer.get_feature_names()

print(len(feature_names))
print(feature_names)
#print(train_vector_data[10])

1494
['00', '10', '100', '12', '15', '17', '1979', '20', '2007', '23', '30', '35', '40', '40min', '45', '4ths', '5lb', '70', '90', 'absolute', 'absolutely', 'absolutley', 'accident', 'accommodations', 'accomodate', 'accordingly', 'accountant', 'ache', 'acknowledged', 'actual', 'actually', 'added', 'afternoon', 'ago', 'ahead', 'airline', 'airport', 'almonds', 'amazing', 'ambiance', 'ambience', 'ample', 'andddd', 'angry', 'annoying', 'anytime', 'anyways', 'apart', 'apology', 'app', 'appalling', 'apparently', 'appetizer', 'appetizers', 'apple', 'area', 'aria', 'array', 'arrived', 'arriving', 'ask', 'asked', 'asking', 'assure', 'ate', 'atmosphere', 'atrocious', 'attached', 'attack', 'attention', 'attentive', 'attitudes', 'auju', 'authentic', 'average', 'avocado', 'avoid', 'away', 'awesome', 'awkward', 'awkwardly', 'ayce', 'az', 'baba', 'baby', 'bachi', 'bacon', 'bad', 'bagels', 'bakery', 'baklava', 'ball', 'bamboo', 'banana', 'bank', 'bar', 'barely', 'bargain', 'bars', 'bartender', 'baseba

In [158]:
import numpy as np
import numpy.ma as ma

def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data:
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')

In [159]:
#write_terms(feature_names, train_data.data, train_vector_data, 10)
write_terms(feature_names, None, train_vector_data, 0)
write_terms(feature_names, None, train_vector_data, 10)

write_terms(feature_names, None, train_vector_data, 100)
write_terms(feature_names, None, train_vector_data, 200)

Mensaje 0 vectorizado: ['loved' 'place' 'wow'] 

Mensaje 10 vectorizado: ['prompt' 'service'] 

Mensaje 100 vectorizado: ['added' 'bone' 'extra' 'fantastic' 'garlic' 'loves' 'marrow' 'meal'
 'roasted' 'server' 'wife'] 

Mensaje 200 vectorizado: ['dreamed' 'exceeding' 'good' 'heard' 'hope' 'place' 'things'] 



In [160]:
vectorizer = CountVectorizer(vocabulary=lines_org, stop_words='english')    

In [163]:
feature_names = vectorizer.get_feature_names()

print(len(feature_names))

ValueError: Duplicate term in vocabulary: 'I love this place.\t1'

In [164]:
tfidfer = TfidfTransformer()
train_preprocessed = tfidfer.fit_transform(train_vector_data)

In [104]:
#write_terms(feature_names, train_data.data, train_vector_data, 10)
write_terms(feature_names, None, train_vector_data, 10)

write_terms(feature_names, None, train_vector_data, 100)

Mensaje 10 vectorizado: ['prompt' 'service'] 

Mensaje 100 vectorizado: ['added' 'bone' 'extra' 'fantastic' 'garlic' 'loves' 'marrow' 'meal'
 'roasted' 'server' 'wife'] 



In [105]:
# Tomamos los textos del conjunto de test y los transformamos en una matriz
# de palabras. Al usar "transform" toma como referencia únicamente las palabras
# encontradas en el conjunto de entrenamiento
test_vector_data=vectorizer.transform(test_data.data)
# Calculamos el valor TF-IDF 
# Al usar "transform" toma como IDF el del conjunto de entrenamiento 
test_preprocessed=tfidfer.transform(test_vector_data)

NameError: name 'test_data' is not defined



    
#### Para cada una de esas 4 combinaciones entrenaremos dos clasificadores:

     1. Un clasificador naive bayes, eligiendo el más adecuado para cada caso.
     
     2. Un árbol de decisión buscando un valor óptimo para uno de los siguientes parámetros para que se maximice la tasa de aciertos en el conjunto de test: max_depth, min_samples_leaf o max_leaf_nodes (siempre el mismo).
     


#### Analiza la tasa de aciertos de entrenamiento y test de los 2 clasificadores en las 4 representaciones de bolsa de palabras (8 configuraciones en total) y contesta a las siguientes preguntas:

    - ¿Hay un clasificador que sea superior al otro? ¿por qué crees que sucede?
    
    - Para cada clasificador, ¿tiene un efecto positivo el añadir “complejidad” a la vectorización? Es decir, añadir bigramas y añadir tf-idf. ¿Por qué crees que sucede este efecto positivo o la falta del mismo? 
    


#### Selecciona el mejor árbol de decisión y obtén las 25 variables con más poder discriminante:
    
    - ¿Predominan más las palabras de uno u otro sentimiento? ¿por qué? ¿hay ruido? 
    


#### Selecciona el mejor clasificador naive bayes y obtén las 25 variables con más presencia en cada clase:
    
    - ¿Tienen sentido las palabras seleccionadas? ¿hay ruido (palabras sin sentimiento o de sentimiento opuesto al esperado)? ¿por qué crees que suceden estos fenómenos?
    


#### Finalmente, explica de manera razonada las conclusiones que has extraído de todo el estudio realizado en este apartado.






### Apartado b)

#### Toma el mejor clasificador Naive Bayes y el mejor árbol de decisión y analiza a fondo sus resultados en el conjunto de test.

   

#### 1. Analiza la precisión y la exhaustividad de cada clasificador en cada una de las clases (opiniones positivas y negativas).
    
         Para cada clasificador, ¿tiene un comportamiento homogéneo a la hora de clasificar ambas clases?
         ¿Cuáles son las fortalezas y debilidades de cada uno de los clasificadores?
         ¿Hay algún clasificador que sea mejor que el otro en todo?
         ¿Coinciden ambos clasificadores a la hora de clasificar mejor una clase que la otra?
         


#### 2. Pinta los 8 primeros niveles del árbol de decisión y comenta lo que ves.
         ¿Qué estructura tiene el árbol?
         ¿Cómo interpretas los niveles que has pintado? ¿tienen algún sentido con respecto a la tasa de aciertos, o la precisión y exhaustividad del clasificador? o ¿Hay nodos impuros?
         


#### 3. Por cada clasificador identifica 2 críticas que hayan sido falsas positivas (malas críticas calificadas como buenas) y 2 críticas que han sido falsas negativas (buenas críticas clasificadas como malas). Analiza tanto su texto original, como el vector de palabras resultante (solamente los términos activos).
         ¿Por qué crees que ha fallado el clasificador en cada uno de los casos?
         ¿Se te ocurre alguna idea sobre cómo mejorar el clasificador de sentimiento?